# **Required Package Installations**

In [ ]:
!pip install cloud-sql-python-connector

In [ ]:
!pip install pymysql

In [ ]:
project_id="<your GCP project id"
!gcloud config set project {project_id}

Updated property [core/project].


# **Authenticate User**

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from pp_dbconnect import PPDbConnector

conname='<your db connection string>'
user= '<your db login>'
password='<your password>'
db='phonepe-bn'
pp_connector = PPDbConnector(conname,user,password,db)
conn = pp_connector.connect()

Successfully connected to 'phonepe-bn' database!! 


In [ ]:
!pip install -q streamlit
!pip install streamlit-option-menu
!pip install plotly
!pip install plotly_express

In [ ]:
%%writefile pp_slit_app.py
'''
  Phonepe - Streamlit WebApp
'''
import streamlit as st
from streamlit_option_menu import option_menu
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import locale
import plotly.express as px
import plotly
import json
#import requests
from datetime import date

sys.path.append(str(Path('pp_utils.py').resolve()))
from pp_utils import fetch_dash, get_years_list, get_quarters_list, get_states_list
from pp_dbconnect import PPDbConnector

def connect_to_db():
  conname='<your db connection string>'
  user= '<your sql login>'
  password='<your sql password>'
  db='phonepe-bn'
  pp_connector = PPDbConnector(conname,user,password,db)
  conn = pp_connector.connect()
  return conn


# Initialize session state variables
if 'conn' not in st.session_state:
  st.session_state.conn=connect_to_db()

if 'years' not in st.session_state:
  st.session_state.years=get_years_list()

if 'quarters' not in st.session_state:
  st.session_state.quarters=get_quarters_list()

if 'states' not in st.session_state:
  st.session_state.states=get_states_list(st.session_state.conn)

if 'selected_year' not in st.session_state:
  st.session_state.selected_year = None

if 'selected_quarter' not in st.session_state:
  st.session_state.selected_quarter = None

if 'selected_state' not in st.session_state:
  st.session_state.selected_state = None

if 'object_type' not in st.session_state:
  st.session_state.object_type='Transaction'

if 'data' not in st.session_state:
  st.session_state.data=fetch_dash(st.session_state.conn,st.session_state.object_type,st.session_state.selected_year,st.session_state.selected_quarter,st.session_state.selected_state)

def to_quarter_label(quarter):
  return quarter[1]

def to_state_label(state):
  return st.session_state.states['State'][state-1]

@st.cache_data
def load_geojson(f):
  return json.load(open(f, "r"))

# Use full width of the page
st.set_page_config(layout='wide')

st.title('PhonePe:')
option_menu(key='object_type', menu_title=None, options=['Transaction','Insurance', 'User'], orientation='horizontal')

#Get dashboard
st.session_state.data = fetch_dash(st.session_state.conn,st.session_state.object_type,st.session_state.selected_year,st.session_state.selected_quarter,st.session_state.selected_state)

dashboard=st.container()
with dashboard:
  # Title
  st.subheader(f'''{st.session_state.object_type}''')

  # Filters
  filters=st.container()
  with filters:
    st.subheader('Timeline')
    _year, _quarter, _state,_ = st.columns([20,20,30,30])
    _year.selectbox('Year', options=st.session_state.years, key='selected_year', index=0, placeholder='Choose Year')
    _quarter.selectbox('Quarter', options=st.session_state.quarters, key='selected_quarter', format_func=to_quarter_label, placeholder='Choose Quarter' )

  # Map
  if 'by_geo' in st.session_state.data and st.session_state.data['by_geo']['charts'][0]['data'].shape[0] >0:
    df=st.session_state.data['by_geo']['charts'][0]['data']
    metric_cols = df.drop(columns=['State Code', 'State']).columns.to_list()
    states_india = load_geojson("states_india.geojson")
    fig = px.choropleth(df,
                        locations="State",
                        featureidkey="properties.st_nm",
                        geojson=states_india,
                        color=metric_cols[0],
                        hover_name='State',
                        hover_data=metric_cols,
                        color_continuous_scale='Inferno_r',
                        center={"lat":23.26, "lon":77.41},
                        fitbounds='locations',
                        scope='asia',
                        title=f"{metric_cols[0]} By State",
                        height=1500)

    fig.update_layout(height=700, width=950, font=dict(family="Arial, Helvetica, sans-serif",size=24,), title_x=0.5)
    fig.update_layout(coloraxis_colorbar_x=-0.25,)
    fig.update_geos(fitbounds='locations', visible=False)
    st.plotly_chart(fig,user_container_width=True)
  else:
    st.write('Data not available')

  # Grand Totals
  if 'grand_total' in st.session_state.data:
    gt_view = st.container()
    gt = st.session_state.data['grand_total']
    gt_charts = gt['charts']
    with gt_view:
      st.subheader(f'''{gt['name']}''')
      gtc1, gtc2, _ = st.columns([25,25,50])
      gt1 = gtc1.metric(gt_charts[0]['title'], value=gt_charts[0]['value'])
      gt2 = gtc2.metric(gt_charts[1]['title'], value=gt_charts[1]['value'])

  if 'det_total' in st.session_state.data:
    det_view = st.container()
    det_total= st.session_state.data['det_total']
    det_charts=det_total['charts']
    d=dict()
    d['x']=list()
    d['y']=list()
    for i in range(len(det_charts)):
      d['x'].append(det_charts[i]['title'])
      d['y'].append(det_charts[i]['value'])

    df=pd.DataFrame(d)
    with det_view:
      st.subheader(f'''{det_total['name']}''')
      fig,ax= plt.subplots(figsize=(27,3))
      sns.barplot(data=df,x='x', y='y',ax=ax,width=0.9,hue='x')
      st.pyplot(fig)

  # Top
  if 'top' in st.session_state.data:
    top_view = st.container()
    top = st.session_state.data['top']
    top_charts=top['charts']
    with top_view:
      # Subheader for top chart section
      st.subheader(f'''{top['name']}''')
      fig, (a1,a2,a3) = plt.subplots(figsize=(27,3), ncols =3)
      sns.barplot(data=top_charts[0]['data'],x=top_charts[0]['data'].columns[2], y=top_charts[0]['data'].columns[0], ax=a1,orient='h', width=0.3)
      a1.set_title(top_charts[0]['title'], fontsize = 15)
      a1.set_ylabel(top_charts[0]['data'].columns[0])
      a1.set_yticklabels(a1.get_yticklabels(), rotation=45, ha="right")

      sns.barplot(data=top_charts[1]['data'], x=top_charts[1]['data'].columns[2], y=top_charts[1]['data'].columns[0], ax=a2, orient='h',width=0.3)
      a2.set_title(top_charts[1]['title'], fontsize = 15)
      a2.set_ylabel(top_charts[1]['data'].columns[0])
      a2.set_yticklabels(a2.get_yticklabels(), rotation=45, ha="right")

      sns.barplot(data=top_charts[2]['data'], x=top_charts[2]['data'].columns[2], y=top_charts[2]['data'].columns[0], ax=a3, orient='h',width=0.3)
      a3.set_title(top_charts[2]['title'], fontsize = 15)
      a3.set_ylabel(top_charts[2]['data'].columns[0])
      a3.set_yticklabels(a3.get_yticklabels(), rotation=45, ha="right")
      st.pyplot(fig)




Overwriting pp_slit_app.py


In [ ]:
! wget -q -O - ipv4.icanhazip.com

34.83.79.138


In [ ]:
!streamlit run /content/pp_slit_app.py &>/content/logs.txt &
!npx localtunnel --port 8501

your url is: https://tall-buses-burn.loca.lt
^C
